# Housing prices in Hyderabad, India

## Project Objective 🎯

The objective of this project is to develop a regression model to predict housing prices in Hyderabad, India. Using features such as the property's area, location, number of bedrooms, and available amenities, the model will aim to estimate the market value of a property as accurately as possible.

- This predictive model will be a valuable tool for:
- Home Buyers and Sellers: To obtain an objective price estimate for a property.
- Real Estate Agents: To assist with property valuation and client advisory.
- Investors: To identify potentially undervalued or overvalued properties in the market.

In [41]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA
from IPython.display import display, Markdown
import sys

sys.path.append('../../src/utils')


# Utilities
from regresion_metrics import show_model_equation, get_model_coeficients_dataframe


training_features = pd.read_parquet('../../datasets/processed/housing_prices/hyderabad_house_price_training_features.parquet')
training_labels = pd.read_parquet('../../datasets/processed/housing_prices/hyderabad_house_price_training_labels.parquet')

test_features = pd.read_parquet('../../datasets/processed/housing_prices/hyderabad_house_price_test_features.parquet')
test_labels= pd.read_parquet('../../datasets/processed/housing_prices/hyderabad_house_price_test_labels.parquet')

target_metrics = '../../datasets/processed/housing_prices/hyderabad_house_price_metrics_summary.parquet'

## 1.1 Training default regresion model using cross validation

**Problem:**

We need to train the model, but we want to ensure that our training set is sufficiently representative. Furthermore, we need to obtain a reliable and stable estimate of the model's performance, as a single data split can lead to misleading results (either too optimistic or too pessimistic).

**Justification:**

Cross-validation is used to address this problem. By dividing the data into multiple folds (k) and iteratively training and validating on different subsets, we obtain a more robust measure of the model's generalization ability. However, the choice of k itself can influence the stability and bias of the metrics. A very low k can have high bias, while a very high k can have high variance. Therefore, it is justified to experiment with different values of k to understand how this parameter affects the perceived performance of our model (measured by R² and RMSE).

**Action:**

- We will train a LinearRegression model using a Pipeline.
- We will use the cross_validate function to evaluate its performance with different numbers of folds: 2, 5, 10, and 100.
- For each run, we will calculate the average R-squared (R²) and Root Mean Squared Error (RMSE).
- Finally, we will compile all the results into a single DataFrame to compare how the choice of folds affects the metrics and their standard deviation. This will help us choose a reliable cross-validation strategy.


In [42]:
from dataclasses import dataclass # Necesitas esta clase para el esquema de métricas

@dataclass
class RegresionMetric:
    """Clase para almacenar los resultados de la validación cruzada y retornarlos como diccionario."""
    model_name: str
    r2_mean: float
    r2_std: float
    rmse_mean: float
    rmse_std: float

    def to_dict(self) -> dict:
        """Retorna los atributos de la instancia como un diccionario."""
        return {
            'Model Name': self.model_name,
            'R2 Mean': self.r2_mean,
            'R2 Std': self.r2_std,
            'RMSE Mean': self.rmse_mean,
            'RMSE Std': self.rmse_std
        }

In [43]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from IPython.display import display, Markdown


fold = 10

pipe = Pipeline([
    ('regresion', LinearRegression())
])

scoring_metrics = {
    'neg_rmse': 'neg_root_mean_squared_error',
    'r2': 'r2'
}

cross_validate_metrics = cross_validate(estimator=pipe,
            X=training_features,
            y=training_labels,
            cv=fold ,
            scoring=scoring_metrics)

metrics = RegresionMetric("Default Lineal Regresion",
                cross_validate_metrics['test_r2'].mean(),
                cross_validate_metrics['test_r2'].std(), 
                -cross_validate_metrics['test_neg_rmse'].mean(),
                cross_validate_metrics['test_neg_rmse'].std())

metrics.to_dict() 

{'Model Name': 'Default Lineal Regresion',
 'R2 Mean': np.float64(0.865983780086094),
 'R2 Std': np.float64(0.04510579488639769),
 'RMSE Mean': np.float64(0.2313167010538074),
 'RMSE Std': np.float64(0.03413525472603451)}

### 1.3 Dimensionality Reduction

Problem:

Having too many features (high dimensionality) causes models to overfit, become unstable due to redundant data (multicollinearity), and require significant computational resources to train.

Justification:

PCA reduces the number of features by creating a smaller set of new, uncorrelated features called principal components. This method retains most of the original data's important information (variance) while making the model simpler, faster, and less prone to overfitting.

Action:

- Iterate and Select the top principal components that explain most of the variance.
- Compare the score of the smallest component reduction and best explanation of the variance and rmse and r2_score of the initially calculated regression.
- Transform the dataset into this new, smaller set of features.

In [44]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from IPython.display import display, Markdown


pipe = Pipeline([
    ('pca', PCA()),
    ('regresion', LinearRegression())
])


max_components = training_features.shape[1]
param_grid = {
    'pca__n_components': range(1, max_components + 1)
}

scoring_metrics = {
    'neg_rmse': 'neg_root_mean_squared_error',
    'r2': 'r2'
}

grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring=scoring_metrics,
    refit='r2',
    cv=5,
    n_jobs=-1
)

grid_search.fit(training_features, training_labels)


results_df = pd.DataFrame(grid_search.cv_results_)

results_df.to_parquet("../../datasets/processed/metrics.parquet")